# Lösen der 1-D Diffusionsgleichung (implizit)

Wenden wir nun das implizite finite Differenzen Schema auf unser 1-D Problem der Wärmediffusionsgleichung an. Zur Wiederholung hier noch einmal das Problem:

## Das Problem

Wir sind an der Änderung der Temperatur $T$ mit der Zeit $t$ interessiert, für eine bestimmte Anfangstemperatur $T_{0}[x,t_{0}]$ und Randbedingungen; z.B. eine Dikeintrusion in der Lithosphäre, mit 
- einer Gesammtlänge $L$ = 100 m, 
- einer Dikebreite $W$ = 5 m, 
- der thermischen Diffusivität $\kappa$ = 10⁻⁶ m²/s, 
- der Diketemperatur $T_{dike}$ = 1200 °C, und 
- einer Hintergrundtempertur $T_{background}$ = 300 °C.

<img src="./Figures/Exercise02_1.png" alt="drawing" width="350"/> <br>
**Abb. 1.** Sketch des geologischen Problem und des Profils der Anfangstemperaturbedingung. 

Wie lange würde es dauern, bis sich der Dike auf eine bestimmte Temperatur abgekühlt hat?

Zur Bestimmung der Dauer der Abkühlung müssen wir die Diffusionsgleichung der Temperatur lösen (parabolische, partielle Differentialgleichung (PDG)):

$\begin{equation}
\rho c_p \frac{\partial{T}}{\partial{t}} = \frac{\partial{}}{\partial{x}} \left( k\frac{\partial{T}}{\partial{x}} \right),
\end{equation}$

wobei $k$ die thermische Leitfähigkeit [W/m/K], $T$ die Temperatur [K] und $\rho$ die Dichte [kg/m³] ist. Durch Umformen (Annahme, dass die thermischen Parameter konstant sind!) können wir die Gleichung durch die thermische Diffusivität $\kappa = \frac{k}{\rho c_p}$ ausdrücken: 

$\begin{equation}
\frac{\partial{T}}{\partial{t}} = \kappa \frac{\partial^2{T}}{\partial{x^2}}.
\end{equation}$

### Finite Differenzen Approximation

Zur numerischen Lösung des Problems mit Hilfe von finiten Differenzen müssen wir zuerst ein numerisches Gitter erstellen (die Diskretisierung):

<img src="./Figures/Exercise02_2.png" alt="drawing" width="600"/> <br>
**Abb. 2.** Numerisches 1-D Gitter für die Diskretisierung der *PDG*. 

Um die Diffusionsgleichung numerisch mit Hilfe des impliziten finiten Differenzen Schema zu lösen müssen wir die *PDG* wie folgt umformulieren:

$\begin{equation}
\frac{T_{I^\textrm{C}}^{n+1}-T_{I^\textrm{C}}^{n}}{\Delta t} = \frac{T_{I^\textrm{E}}^{n+1} - 2T_{I^\textrm{C}}^{n+1} + T_{I^\textrm{W}}^{n+1}}{(\Delta x)^2}.
\end{equation}$

Diese Gleichung können wir umformulieren, so dass wir ein Gleichungssystem erhalten, mit so vielen Gleichungen wie zentralen Gitterpunkten:

$\begin{equation}
-a T_{I^\textrm{W}}^{n+1} + \left(2a + b\right) T_{I^\textrm{C}}^{n+1} - a T_{I^\textrm{E}}^{n+1} = b T_{I^\textrm{C}}^{n},
\end{equation}$

mit 
$\begin{equation}
a=\frac{\kappa}{\Delta{x^2}}, b = \frac{1}{\Delta{t}}.
\end{equation}$

D.h. wir haben ein tridiagonales (drei Diagonalen) Gleichungssystem, welches durch eine Koeffizientenmatrix $\mathbf{A}$, einem unbekannten Vektor $T^{n+1}$ und einen bekannten Vektor $T^n$ beschrieben werden kann:

$\begin{equation}
\mathbf{K} \cdot \bm{x} = \bm{b}
\end{equation}$

wobei $\mathbf{K}$ die Koeffizientenmatrix (mit drei von Null verschiedenen Diagonalen) ist, $\bm{x}$ der unbekannte Lösungsvektor (d.h. die Temperatur zum Zeitschritt $n+1$) und $\bm{b}$ die bekannte rechte Seite.

#### Allgemeine Lösung

Ein allgemeiner Ansatz zur Lösung dieses Gleichungssystems ist die **Defektkorrektur** (*defect correction*). Die Wärme-Diffusionsgleichung wird durch Einführung eines Residualterms $\bm{r}$ umformuliert, der die Abweichung von der exakten Lösung quantifiziert und iterativ durch sukzessive Korrekturschritte reduziert werden kann, um die Genauigkeit zu verbessern. In impliziter Form kann Gleichung (6) umgeschrieben werden zu:

$\begin{equation}
\mathbf{K} \cdot \bm{x} - \bm{b} = \bm{r},
\end{equation}$

wobei $\bm{r}$ das Residuum (oder der Defekt) ist. Die Koeffizienten der Matrix sind dieselben wie oben hergeleitet (Gleichung (5)). Im Rahmen der Defekkorrektur kann die Koeffizientenmatrix, für ein 1D Problem mit konstanten thermischen Parametern, mit Hilfe der eingebauten Funktion `AssembleMatrix1Dc!()` aufgebaut werden. 

Der Korrekturterm ist gegeben durch:

$\begin{equation}
\delta \bm{T} = -\mathbf{K}^{-1} \bm{r}^k.
\end{equation}$

Damit ergibt sich die aktualisierte Lösung zu:

$\begin{equation}
\bm{T}^{k+1} = \bm{T}^k + \delta \bm{T},
\end{equation}$

wobei $\bm{T}^{k+1}$ die aktualisierte Temperatur nach einem Iterationsschritt ist.

---

In `GeoModBox.jl` wird das Residuum $\bm{r}$ auf den Centroids berechnet, wobei das erweiterte Temperaturfeld inklusive der Ghost Nodes des aktuellen Zeitschritts als Anfangsschätzung verwendet wird:

$\begin{equation}
\frac{\partial{T_{\textrm{ext,I}}}}{\partial{t}} - \kappa \frac{\partial^2{T_{\textrm{ext,I}}}}{\partial{x^2}} - \frac{Q}{\rho_0 c_p} = r_{I},
\end{equation}$

und in diskretisierter, impliziter Finite-Differenzen-Form:

$\begin{equation}
\frac{T_{\textrm{ext},I^\textrm{C}}^{n+1} - T_{\textrm{ext},I^\textrm{C}}^{n}}{\Delta{t}} - \kappa \frac{T_{\textrm{ext},I^\textrm{W}}^{n+1} - 2 T_{\textrm{ext},I^\textrm{C}}^{n+1} + T_{\textrm{ext},I^\textrm{E}}^{n+1}}{\Delta{x^2}} - \frac{Q_{I^\textrm{C}}^n}{\rho_0 c_p}= r_{I},
\end{equation}$

wobei $I^\textrm{C}=2:(nc+1)$ der Index der Centroids im erweiterten Temperaturfeld ist und $I=1:nc$ die Gleichungsnummer bezeichnet. Das Umformen von Gleichung (11) und das Einsetzen der Koeffizienten mittels Gleichung (5) ergibt:

$\begin{equation}
-aT_{\textrm{ext},I^\textrm{W}}^{n+1}
+\left(2a+b\right)T_{\textrm{ext},I^\textrm{C}}^{n+1}
-aT_{\textrm{ext},I^\textrm{E}}^{n+1}
-bT_{\textrm{ext},I^\textrm{C}}^n
-\frac{Q_{I^\textrm{C}}^n}{\rho_0 c_p}=r_{I},
\end{equation}$

was der Matrixform von Gleichung (7) entspricht, wobei $T_{\textrm{ext},I^\textrm{C}}^{n+1}$ der unbekannte Vektor $x$ ist, $-bT_{\textrm{ext},I^\textrm{C}}^n -\frac{Q_{I^\textrm{C}}^n}{\rho_0 c_p}$ der bekannte Vektor $b$ ist und $-a$ sowie $2a+b$ die Koeffizienten der von Null verschiedenen Diagonalen der Koeffizientenmatrix sind. 

Das Residuum kann mit Hilfe der eingebauten Funktion `ComputeResiduals1Dc!()` berechnet werden. Mit dem Residualvektor $\bm{r}$ und der Koeffizientenmatrix $\bm{K}$ kann der Korrekturterm für die Temperatur über Gleichung (8) berechnet werden.

#### Spezialfall – Ein lineares Problem

Wenn das Problem linear ist und die exakte Lösung innerhalb eines einzigen Iterationsschritts erreicht wird, reduziert sich das Gleichungssystem auf Gleichung (6). Damit kann das Gleichungssystem direkt über eine *linke Matrixdivision* gelöst werden:

$\begin{equation}
\bf{x} = \mathbf{K}^{-1} \bm{b}.
\end{equation}$

Die Koeffizientenmatrix bleibt dabei unverändert, auch für die gegebenen Randbedingungen. Allerdings muss die rechte Seite entsprechend aktualisiert werden (Setzen von $\bm{r}=0$ und Hinzufügen der bekannten Parameter zur rechten Seite der Gleichungen).

Hier wollen wir zuerst die spezielle Lösung formulieren, bevor wir dei Verwendung des generllen, eingebauten Solvers anwenden. 

Für mehr Informationen zur Lösung der PDG mit Hilfe einer impliziten finiten Differenzendiskretisierung und den entsprechenden Bedingungen, sowie Vor- und Nachteilen, überprüft bitte die entsprechende [Dokumentation](https://geosci-ffm.github.io/GeoModBox.jl/dev/man/DiffOneD).

Laden wir nun zuerst die notwendigen Module zur numerischen Lösung unseres Problems. Zuerst wollen wir die Solver selbst programmieren, bevor die voreigestellen Solver der `GeoModBox.jl` genutzt werden können. Der voreigestelle Solver befindet sich im Submodul `GeoModBox.HeatEquation.OneD`. 

In [ ]:
using Plots, ExtendableSparse, LinearAlgebra, Printf
using ?

### Parameterdefinitionen

Definieren wir für das Problem erst einmal bestimmte Parameter (physikalische Konstanten, numerische Domaine, Zeitparameters). 

Der Parameter `alternative` beschreibt unterschiedliche Alternativen in der Programmierung. 
1) Vollständig ausformulierte Lösung (Spezialfall)
2) Verwendung der eingebauten Funktion (Spezialfall)
3) Verwendung der generellen Lösung mit Hilfe der "*defection correction*" 

In [ ]:
alternative =   1

# Physikalische Parameter ----------------------------------------------- #
L           =   ?       # Laenge der Modeldomain [m]
Tdike       =   ?       # Temperatur des Dikes [C]
Trock       =   ?       # Temperatur des Umgebungsgesteins [C]
κ           =   ?       # Thermische Diffusivitaet des Gesteins [m2/s]
W           =   ?       # Breite des Dykes [m]
# ----------------------------------------------------------------------- #
# Numerische Parameter -------------------------------------------------- #
nc          =   ?               # Anzahl der Gitterpunkte in x-Richtung
Δx          =   ?               # Gitterlaenge
xc          =   ?               # x-Koordinaaten den Centroids
# Iterations --  defection correction method
niter       =   50  
ϵ           =   1.0e-15       
# ----------------------------------------------------------------------- #
# Zeit Parameter -------------------------------------------------------- #
day         =   3600.0*24.0     # Sekunden pro Tag
fac         =   5.0             # Multiplikationsfaktor für dt
Δt          =   ?               # Zeitschrittlänge 
tmax        =   2.0*365.0*day 
nt          =   ceil(Int,tmax/Δt)
time        =   0.0
# ----------------------------------------------------------------------- #

### Anfangsbedingungen

Zur Lösung unseres Problem, müssen wir noch die Anfangsbedingungen definieren. Dazu nehmen wir 300 °C für das Umgebungsgestein und 1200 °C für den Dike an. D.h. die Anfangstemperatur ist definiert durch:

$\begin{equation}
T \left(x < \left( \frac{L}{2} - W \right), x > \left( \frac{L}{2} + W \right), t = 0 \right) = 300,
\end{equation}$
$\begin{equation}
T \left(x > \left( \frac{L}{2} - W \right), x < \left( \frac{L}{2} + W \right), t = 0 \right) = 1200.
\end{equation}$

Jetzt wollen wir erst einmal die Anfangsbedingung graphisch darstellen. 

In [ ]:
# Anfangsbedingungen; Temperaturprofil ---------------------------------- #
T   =   (
            T       =   zeros( ? ),     # Temperatur auf den Centroids 
            T0      =   zeros( ? ),     # Altes Temperaturfeld
            T_ex    =   zeros( ? ),     # inklusive Ghost Nodes
            T_ex0   =   zeros( ? ),     # Altes Temperaturfeld
            R       =   zeros( ? ),     # Residuum
)
∂2T =   (                               
        ∂x2     =   zeros( ? ),         # 2. Ableitung von T im Raum
        ∂x20    =   zeros( ? ),         # Altes Feld
)
T.T        .=  ?   # Temperatur des Umgebungsgesteins
@. T.T[?]  =   ?   # Temperatur des Dikes
T.T_ex[2:end-1]         .=      T.T
T.T0                    .=      T.T
# ----------------------------------------------------------------------- #
# Plot initial condition ------------------------------------------------ #
p = plot( ? )

display(p)
# ----------------------------------------------------------------------- #

### Randbedingungen

Da wir zentrale Gitterpunkte (Centroids) für die Temperatur verwenden, liegt kein Gitterpunkt direkt auf den Rändern (hingegen jeder Erwartung ist das allerdings ein Vorteil!). Zur Festlegung der Temperaturrandbedingungen müssen wir uns also der zusätzlichen *Ghost nodes* bediehnen (siehe Abbildung 2), d.h. wir bestimmen die Temperatur auf den *Ghost nodes* um die partielle Differentialgleichung der Temperatur auf dem **nächsten inneren** Gitterpunkt mit Hilfe der finiten Differenzen lösen zu können. Für eine konstante Temperaturbedingung and den Rändern (Dirichlet), können wir die Temperatur der *Ghost nodes* durch lineare Interpolation bestimmem, so dass: 

**West**
$\begin{equation}
T_{Ghost}^W = 2 T_{BC}^W + T_{1},
\end{equation}$
**East**
$\begin{equation}
T_{Ghost}^E  = 2 T_{BC}^E + T_{nc}.
\end{equation}$

Für konstante Flussrandbedingungen ist die Temperatur an den Rändern gegeben durch: 

**West**
$\begin{equation}
T_{Ghost}^W = T_{1} - c^W \Delta x,
\end{equation}$
**East**

$\begin{equation}
T_{Ghost}^E = T_{1} + c^E \Delta x,
\end{equation}$

wobei $c^W = \frac{\partial{T}}{\partial{x}}$ und $c^E = \frac{\partial{T}}{\partial{x}}$ die Flussbedingungen am jeweiligen Rand definieren. 

Im Falle des generellen Solvers der Defektkorrektur, werden die Randbedingungen eingebunden indem man das Residuum der äußeren Centroids berechnet mit Hilfe der Temperatur auf den Ghost Nodes. D.h. abgesehen der Änderungen der Koeffizienten der zugehörigen Centroids müssen keine speziellen Änderungen vorgenommen werden. 

Für den Spezialfall müssen die Koeffizienten und die rechte Seite des Gleichungssystems für die Gleichungen des **ersten** und **letzten inneren Gitterpunktes** in Abhängigkeit der Randbedingungen wie folgt modifizieren werden: 

#### **Dirichlet**
*West*
$\begin{equation}
\left(3 a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^{n} + 2 a T_{BC}^W
\end{equation}$
*East*
$\begin{equation}
-a T_{nc-1}^{n+1} + \left(3 a + b \right) T_{nc}^{n+1} = b T_{nc}^{n} + 2 a T_{BC}^E
\end{equation}$

#### **Neumann**
*West*
$\begin{equation}
\left(a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^n - a c^{W} \Delta{x}
\end{equation}$
*East*
$\begin{equation}
-a T_{nc-1}^{n+1} + \left(a + b \right) T_{nc}^{n+1}  = b T_{nc}^n - a c^{E} \Delta{x}
\end{equation}$

Bei der Initialisierung der Randbedingungen nutzen wir die Hilfe des Tuples `BC`. Das Tuple definierte die Art der Randbedingung (`type`) und den dafür benötigten Wert (`val`). In der Lösung verwenden wir if-Abfragen und logische Parameter um die Koeffizienten der Koeffizienten Matrix und rechte Seite des Gleichungssystems zu modifizieren. 

In [ ]:
# Randbdingungen -------------------------------------------------------- #
BC   = (
    type = (W=:Dirichlet, E=:Dirichlet),
    # type = (W=:Neumann, E=:Neumann),
    val  = (W=300.0, E=300.0))
# ----------------------------------------------------------------------- #

### Gleichungssystem

Lasst uns nun die Koeffizientenmatrix und rechte Seite initialisieren: 

In [ ]:
# Aufbauen der Koeffizientenmatrix -------------------------------------- #
# Definition der Matrix (hier in sparsamer Form)
ndof        =   length(?)                       # Anzahl der Freiheitsgrade
K           =   ExtendableSparseMatrix(?,?)     # Koeffizientenmatrix
rhs         =   zeros(?)                        # Rechte-Seite-Vektor
# ----------------------------------------------------------------------- #

### Visualisierung

Zur Visualisierung der Ergebnisse als eine Animation in einer GIF Datei, müssen noch der Ort und der Name der Datei festgelegt werden: 

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("03_1D_implicit_",alternative)
save_fig    =   1
# ----------------------------------------------------------------------- #          

### Zeitschleife

Numerisch können wir nun in einer Zeitschleife die *PDG* auf unterschiedliche Art und Weise lösen (Alternative I ist ausreichend; wer möchte, kann auch die Alternativen II und III lösen). 

Zur Implementierung der Randbedingungen, benutzen wir die logischen Parameter `DirW, NeuW, DirE,NeuE` sowie die Parameter `inE` und `inW`. Letztere beschreiben, ob ein Koeffizient für die Nebendiagonalen (*West* oder *East*) benötigt wird (`true`) oder nicht (`false`). 

Die logischen Parameter sind für den jeweiligen Rand entweder 0 (falsch) oder 1 (wahr), je nach gewählter Randbedingung (Dirichlet oder Neumann). Entsprechend können wir die Parameter nutzen um die Koeffizienten und die rechte Seite zu modifizieren, z.B. (Hauptdiagonale und rechte Seite, Dirichlet, West; siehe Gleichung (20)): 

$$
\left((2+\textrm{DirW}) \cdot a + b\right) \cdot T_1^{n+1} - a T_{2}^{n+1} = b T_{1}^{n} + \textrm{DirW} \cdot (2 a T_{BC,W}).
$$

Im Loop über das Gitter können wir eine allgemeine Formulierung aufstellen, welche alle Randbedingungen für alle Randpunkte berücksichtigt. 

In [ ]:
# Timestep loop --------------------------------------------------------- #
for n = 1:nt
    println("Zeitschritt: ",n,", Time: $(round(time/day, digits=1)) [d]")
    if alternative == 1
        a   =   ?
        b   =   ?

        @. rhs     =   ?

        # Alternative I
        for i = 1:nc  
            # Equation number
            ii          =   i
            # Stencil 
            iW          =   ii - 1
            iC          =   ii
            iE          =   ii + 1   
            # Boundaries 
            inW    =  i==1    ? false  : true
            DirW   = (i==1    && BC.type.W==:Dirichlet) ? 1. : 0.
            NeuW   = (i==1    && BC.type.W==:Neumann  ) ? 1. : 0.
            inE    =  i==nc ? false  : true
            DirE   = (i==nc && BC.type.E==:Dirichlet) ? 1. : 0.
            NeuE   = (i==nc && BC.type.E==:Neumann  ) ? 1. : 0.
            if inE
                K[ii,iE]    = ?
            end
            K[ii,iC]        =  ?
            if inW 
                K[ii,iW]    = ?
            end
            # Aenderung der rechten Seite durch die Randbedingungen ------------- #        
            rhs[i]  += ? 
        end            
        T.T     .=  ?
    elseif alternative == 2            
        BackwardEuler1Dc!( ? )
    elseif alternative == 3
        for iter = 1:niter
            # Residual iteration
            ComputeResiduals1Dc!( ? )
            @printf("||R|| = %1.4e\n", norm(T.R)/length(T.R))            
            norm(T.R)/length(T.R) < ϵ ? break : nothing
            # Assemble linear system
            AssembleMatrix1Dc!(?)
            # Solve for temperature correction: Cholesky factorisation
            Kc = cholesky(K.cscmatrix)
            # Solve for temperature correction: Back substitutions
            δT  = ?             
            # Update temperature            
            T.T .=  ?
        end        
        # Überschreiben wir nur das alte Temperaturfeld mit dem neuen      
        @. T.T0     =   ? 
    end    
    # Berechnung der Zeit ---
    time    =   time + Δt        
    # Plot Lösung ---
    p = plot( ? )
    if save_fig == 1
        Plots.frame(anim)
    else
        display(p)
    end
end

Nun müssen wir noch die Animation erstellen und speichern:

In [ ]:
# Speicher Animation ---------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
else
    display(p)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
# ----------------------------------------------------------------------- #